In [1]:
import geopandas
import pandas as pd
import xarray as xr
from config import get_config
config = get_config()

## Fusion des communes en bassin de vie

In [2]:
communes_geo = geopandas.read_file(config['path_communes'])
type(communes_geo)

geopandas.geodataframe.GeoDataFrame

In [4]:
communes_geo.shape
communes = pd.read_csv(config['path_BdV'], sep =';', encoding='latin1', dtype=str) 
commune_bdv = pd.merge(communes_geo, communes, left_on = 'insee', right_on = 'CODGEO', how = 'inner').drop('CODGEO', axis = 1)
print(commune_bdv.shape)
commune_bdv.head()

(34970, 10)


,insee,nom,wikipedia,surf_ha,geometry,LIBGEO,BV2012,LIBBV2012,DEP,REG
0,97223,Saint-Esprit,fr:Saint-Esprit (Martinique),2318.0,"POLYGON ((-60.95617 14.55766, -60.95561 14.557...",Saint-Esprit,97222,Le Robert,972,02
1,97233,Le Morne-Vert,fr:Le Morne-Vert,1325.0,"POLYGON ((-61.15922 14.70645, -61.15917 14.706...",Le Morne-Vert,97209,Fort-de-France,972,02
2,97208,Fonds-Saint-Denis,fr:Fonds-Saint-Denis,2374.0,"POLYGON ((-61.16254 14.74252, -61.16232 14.743...",Fonds-Saint-Denis,97209,Fort-de-France,972,02
3,97224,Saint-Joseph,fr:Saint-Joseph (Martinique),4324.0,"POLYGON ((-61.08680 14.71891, -61.08678 14.719...",Saint-Joseph,97209,Fort-de-France,972,02
4,97212,Gros-Morne,fr:Gros-Morne,4601.0,"POLYGON ((-61.08483 14.72449, -61.08481 14.724...",Gros-Morne,97222,Le Robert,972,02


In [5]:
bassin_de_vie = commune_bdv.dissolve(by='BV2012')

In [6]:
bassin_de_vie.to_file(config['path_BdV_shp'])

## Concaténation de la pollution sur les bassins de vie

In [7]:
ds = xr.open_dataset(config['path_PM25'])
df = ds.to_dataframe()

In [8]:
ds.head()

<xarray.Dataset>
Dimensions:     (Times: 5, lat: 5, lon: 5, bnds: 2)
Coordinates:
  * Times       (Times) float64 2.019e+07 2.019e+07 ... 2.019e+07 2.019e+07
  * lat         (lat) float64 40.48 40.49 40.51 40.52 40.54
  * lon         (lon) float64 -5.578 -5.547 -5.516 -5.484 -5.453
Dimensions without coordinates: bnds
Data variables:
    PM25        (Times, lat, lon) float32 nan nan nan nan ... nan nan nan nan
    Times_bnds  (Times, bnds) float64 2.019e+07 2.019e+07 ... 2.019e+07
Attributes:
    CDI:                       Climate Data Interface version 1.7.2 (http://m...
    Conventions:               CF-1.4
    Title:                     Reanalysed CHIMERE data
    Generating_process:        Generated by post-processing (kriging) of CHIM...
    Domain:                    FRA02e
    Chimere_version:           V2017
    NCO:                       "4.6.0"
    nco_openmp_thread_number:  1
    frequency:                 day
    CDO:                       Climate Data Operators version 1.7.2rc6 (http:...

In [9]:
print(type(ds))
print(ds.dims)

<class 'xarray.core.dataset.Dataset'>
Frozen({'Times': 365, 'lat': 761, 'lon': 541, 'bnds': 2})


In [10]:
print('number of longitudinal values :',len(ds.variables['lon']))
print('number of lattitude values :',len(ds.variables['lat']))

number of longitudinal values : 541
number of lattitude values : 761


In [11]:
ds.variables['PM25'].shape

(365, 761, 541)